# 🎯 SAM2 파인튜닝 튜토리얼

이 노트북은 SAM2(Segment Anything Model 2)를 파인튜닝하는 과정을 단계별로 안내합니다.

**목표**: 적은 데이터와 리소스로 특정 도메인에 맞는 세분화 모델 만들기

## 1. 환경 확인

먼저 필요한 패키지가 설치되어 있는지 확인합니다.

In [ ]:
# 환경 확인
import sys
print(f"Python 버전: {sys.version}")

# PyTorch 확인
try:
    import torch
    print(f"PyTorch 버전: {torch.__version__}")
    print(f"MPS (Apple Silicon) 사용 가능: {torch.backends.mps.is_available()}")
    print(f"CUDA (NVIDIA) 사용 가능: {torch.cuda.is_available()}")
except ImportError:
    print("❌ PyTorch가 설치되지 않았습니다.")
    print("설치: pip install torch torchvision")

# SAM2 확인
try:
    from sam2.build_sam import build_sam2
    print("✅ SAM2 설치 확인됨")
except ImportError:
    print("❌ SAM2가 설치되지 않았습니다.")
    print("설치: pip install git+https://github.com/facebookresearch/sam2.git")

## 2. 데이터 폴더 구조 생성

In [ ]:
# 데이터 폴더 생성
from pathlib import Path

folders = [
    "data/images/train",
    "data/images/val",
    "data/masks/train",
    "data/masks/val",
    "checkpoints",
    "output"
]

for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)
    print(f"✅ {folder}")

print("\n📁 폴더 구조 생성 완료!")

## 3. SAM2 모델 다운로드

아래 셀을 실행하면 SAM2 체크포인트가 다운로드됩니다.

In [ ]:
import os
import urllib.request

checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt"
checkpoint_path = "checkpoints/sam2.1_hiera_small.pt"

if not os.path.exists(checkpoint_path):
    print("📥 SAM2 체크포인트 다운로드 중... (약 185MB)")
    urllib.request.urlretrieve(checkpoint_url, checkpoint_path)
    print(f"✅ 다운로드 완료: {checkpoint_path}")
else:
    print(f"✅ 체크포인트가 이미 존재합니다: {checkpoint_path}")

## 4. SAM2 모델 로드 테스트

In [ ]:
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

# 디바이스 설정
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🍎 Apple Silicon GPU 사용")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("🎮 NVIDIA GPU 사용")
else:
    device = torch.device("cpu")
    print("💻 CPU 사용")

# 모델 로드
print("\n📦 SAM2 모델 로드 중...")
model = build_sam2("sam2.1_hiera_s", "checkpoints/sam2.1_hiera_small.pt")
model = model.to(device)
predictor = SAM2ImagePredictor(model)
print("✅ 모델 로드 완료!")

## 5. 샘플 이미지로 테스트

실제 파인튜닝 전에 원본 SAM2가 어떻게 작동하는지 확인합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels == 1]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green',
               marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

# 샘플 이미지 생성 (테스트용 - 실제로는 자신의 이미지를 사용하세요)
print("🖼️ 샘플 이미지 생성 중...")
sample_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
# 중앙에 원 그리기 (세분화 대상)
from PIL import ImageDraw
img_pil = Image.fromarray(sample_image)
draw = ImageDraw.Draw(img_pil)
draw.ellipse([220, 140, 420, 340], fill=(255, 100, 100))
sample_image = np.array(img_pil)

# 세분화 테스트
predictor.set_image(sample_image)

# 중앙 포인트 클릭
input_point = np.array([[320, 240]])  # 중앙
input_label = np.array([1])  # foreground

masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True
)

# 결과 시각화
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(sample_image)
show_points(input_point, input_label, axes[0])
axes[0].set_title("Input Image + Point")
axes[0].axis("off")

for i, (mask, score) in enumerate(zip(masks, scores)):
    axes[i+1].imshow(sample_image)
    show_mask(mask, axes[i+1])
    axes[i+1].set_title(f"Mask {i+1} (score: {score:.3f})")
    axes[i+1].axis("off")

plt.tight_layout()
plt.show()

print("\n✅ SAM2 테스트 완료!")
print("💡 이제 실제 이미지를 data/images/train 폴더에 넣고 파인튜닝을 시작하세요.")

## 6. 다음 단계

1. **데이터 준비**: `data/images/train`에 이미지, `data/masks/train`에 마스크 넣기
2. **파인튜닝 실행**: `python finetune_sam2.py`
3. **테스트**: `python test_sam2.py`

자세한 내용은 `setup_guide.md`를 참고하세요.